# Working RAG Pipeline
## All Issues Fixed!

In [1]:
import sys
from pathlib import Path

project_root = str(Path.cwd().parent.parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
from abc import ABC, abstractmethod
from typing import List
import ollama
from src.utils.config import LLMConfig, get_config
from src.shared.models import SearchResult

## Ollama Manager

In [3]:
class OllamaManager:
    @staticmethod
    def ensure_ready(model_name: str = "llama3.2"):
        try:
            models = ollama.list()
            if not any(model_name in m['name'] for m in models['models']):
                print(f"📥 Downloading {model_name}...")
                ollama.pull(model_name)
            print(f"✅ {model_name} ready")
            return True
        except Exception as e:
            print(f"⚠️  Ollama not running!")
            print(f"   Run: ollama serve")
            print(f"   Then: ollama pull {model_name}")
            return False

## Generator

In [4]:
class BaseGenerator(ABC):
    @abstractmethod
    def generate(self, prompt: str) -> str:
        pass

class OllamaGenerator(BaseGenerator):
    def __init__(self, config: LLMConfig, auto_setup: bool = True):
        self.model = config.model_name
        self.temperature = config.temperature
        if auto_setup:
            OllamaManager.ensure_ready(self.model)
    
    def generate(self, prompt: str) -> str:
        try:
            response = ollama.chat(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                options={"temperature": self.temperature}
            )
            return response['message']['content']
        except Exception as e:
            return f"Error: {e}"

## Query Constructor

In [5]:
class QueryConstructor(ABC):
    @abstractmethod
    def refine_query(self, query: str) -> list[str]:
        pass

class MultiQueryConstructor(QueryConstructor):
    def __init__(self, generator: BaseGenerator) -> None:
        self.generator = generator
        self.template = """Generate 3 different versions of this question for better document search.
Original question: {question}

Provide 3 alternative phrasings, one per line:"""

    def refine_query(self, query: str) -> list[str]:
        prompt = self.template.format(question=query)
        response = self.generator.generate(prompt)
        queries = [q.strip() for q in response.split('\n') if q.strip() and len(q.strip()) > 10]
        return [query] + queries[:3]  # Original + up to 3 variations

## Answer Generator

In [6]:
class BaseQueryAnswerer(ABC):
    @abstractmethod
    def answer(self, result_search: List[SearchResult], query: str) -> str:
        pass

class QueryAnswerer(BaseQueryAnswerer):
    def __init__(self, generator: BaseGenerator) -> None:
        self.generator = generator
        self.template = """Answer this question using only the context below.

Context:
{context}

Question: {question}

Answer:"""
    
    def answer(self, result_search: List[SearchResult], query: str) -> str:
        if not result_search:
            return "No relevant documents found."
        
        context_parts = [f"[{i}] {r.content}" for i, r in enumerate(result_search, 1)]
        context = "\n\n".join(context_parts)
        prompt = self.template.format(context=context, question=query)
        return self.generator.generate(prompt).strip()

## RAG Pipeline (SIMPLE VERSION)

In [ ]:
from src.ingestion.vector_store.stores import ChromaStore

class SimpleRAGPipeline:
    def __init__(self, vector_store: ChromaStore, answerer: BaseQueryAnswerer):
        self.vector_store = vector_store
        self.answerer = answerer
    
    def query(self, user_query: str, top_k: int = 5) -> str:
        print(f"🔍 Searching for: {user_query}")
        
        # Simple single query
        results = self.vector_store.query_flattened([user_query], n_result=top_k)
        
        print(f"📄 Found {len(results)} results")
        
        # Generate answer
        answer = self.answerer.answer(results, user_query)
        return answer


class MultiQueryRAGPipeline:
    def __init__(
        self,
        query_constructor: QueryConstructor,
        vector_store: ChromaStore, 
        answerer: BaseQueryAnswerer
    ):
        self.query_constructor = query_constructor
        self.vector_store = vector_store
        self.answerer = answerer
    
    def query(self, user_query: str, top_k: int = 5) -> str:
        # Generate query variations
        queries = self.query_constructor.refine_query(user_query)
        print(f"🔍 Using {len(queries)} query variations")
        
        # Search with all variations
        results = self.vector_store.query(queries, n_result=top_k)
        
        print(f"📄 Found {len(results)} total results")
        
        # Take top results
        top_results = results[:top_k]
        
        # Generate answer
        answer = self.answerer.answer(top_results, user_query)
        return answer

## Test Simple Version First

In [16]:
from src.ingestion.parsers.get_parser import get_parser

In [17]:
parser = get_parser()

In [18]:
pdf_path = Path("../../data/Word2Vec.pdf")

# Verify it exists
import os

print(f"File exists: {os.path.exists(pdf_path)}")

File exists: True


In [19]:
from src.ingestion.chunking.get_chunker import get_chunker

In [20]:
chunker = get_chunker()

In [21]:
parsed_doc = parser.parse(pdf_path=pdf_path)
chunked_doc = chunker.chunk(parsed_doc)

2026-01-29 23:21:09 | INFO     | src.ingestion.parsers.parsers:parse:22 - Starting to parse PDF: ../../data/Word2Vec.pdf
2026-01-29 23:21:19 | INFO     | src.ingestion.parsers.parsers:parse:42 - Document converted successfully: 12 pages
2026-01-29 23:21:19 | INFO     | src.ingestion.parsers.parsers:parse:64 - Structure extracted: 23 chapters
2026-01-29 23:21:19 | SUCCESS  | src.ingestion.parsers.parsers:parse:66 - Successfully parsed Word2Vec.pdf


In [23]:
from src.ingestion.embedding.get_embbedder import get_embedder

In [24]:
embedder = get_embedder()

2026-01-29 23:22:12 | INFO     | src.ingestion.embedding.embedder:__init__:19 - Loading SentenceTransformer model: all-MiniLM-L6-v2
2026-01-29 23:22:19 | INFO     | src.ingestion.embedding.embedder:__init__:27 - Model loaded: 384d on cpu


In [25]:
embeddings = embedder.embed_chunk(chunks=chunked_doc)

2026-01-29 23:22:22 | INFO     | src.ingestion.embedding.base_embed:embed_chunk:62 - Successfully embedded 33 chunks


In [26]:
from src.utils.config import settings

print("🚀 Initializing...")

# Create components
generator = OllamaGenerator(settings.llm, auto_setup=True)
answerer = QueryAnswerer(generator)
vector_store = ChromaStore(settings.vector_store)
vector_store.ingest(embch=embeddings)
# Simple pipeline (no query enhancement)
simple_rag = SimpleRAGPipeline(
    vector_store=vector_store,
    answerer=answerer
)

print("✅ Simple RAG ready!")

2026-01-29 23:22:36 | INFO     | src.ingestion.vector_store.stores:__init__:19 - creating or getting the collection
2026-01-29 23:22:36 | INFO     | src.ingestion.vector_store.stores:__init__:25 - getting the embedder
2026-01-29 23:22:36 | INFO     | src.ingestion.embedding.embedder:__init__:19 - Loading SentenceTransformer model: all-MiniLM-L6-v2


🚀 Initializing...
⚠️  Ollama not running!
   Run: ollama serve
   Then: ollama pull llama3.2


2026-01-29 23:22:44 | INFO     | src.ingestion.embedding.embedder:__init__:27 - Model loaded: 384d on cpu
2026-01-29 23:22:44 | INFO     | src.ingestion.vector_store.stores:ingest:37 - adding chunks to the collection


✅ Simple RAG ready!


In [15]:
vector_store.count()

0

In [27]:
# Test simple version
print("\n" + "="*60)
print("SIMPLE RAG TEST")
print("="*60)

answer = simple_rag.query("What is Word2Vec?", top_k=3)

print("\n📝 Answer:")
print(answer)

2026-01-29 23:22:52 | INFO     | src.ingestion.vector_store.stores:query_flattened:87 - querying the results
2026-01-29 23:22:52 | INFO     | src.ingestion.vector_store.stores:query_flattened:111 - finished the querying



SIMPLE RAG TEST
🔍 Searching for: What is Word2Vec?
📄 Found 3 results


AttributeError: 'SearchResult' object has no attribute 'content'

## Test Multi-Query Version

In [13]:
# Multi-query pipeline (with query enhancement)
query_constructor = MultiQueryConstructor(generator)

multi_rag = MultiQueryRAGPipeline(
    query_constructor=query_constructor,
    vector_store=vector_store,
    answerer=answerer
)

print("✅ Multi-query RAG ready!")

✅ Multi-query RAG ready!


In [14]:
# Test multi-query version
print("\n" + "="*60)
print("MULTI-QUERY RAG TEST")
print("="*60)

answer = multi_rag.query("What is Word2Vec?", top_k=5)

print("\n📝 Answer:")
print(answer)


MULTI-QUERY RAG TEST


2026-01-29 23:18:18 | INFO     | src.ingestion.vector_store.stores:query:47 - quering the results
2026-01-29 23:18:18 | INFO     | src.ingestion.vector_store.stores:query:73 - finished the quering


🔍 Using 4 query variations
📄 Found 4 total results


AttributeError: 'list' object has no attribute 'content'

## Debug: Check what's in the vector store

In [ ]:
print(f"Vector store has {vector_store.count()} documents")

# Try a simple search
test_results = vector_store.query(["word embedding"], n_result=2)
print(f"\nFound {len(test_results)} results for 'word embedding'")

if test_results:
    print("\nFirst result preview:")
    print(test_results[0].content[:200] + "...")